<font size="6">Bold360 and BoldChat API Intergation</font>

In [ ]:
import requests
import json
import time
import hashlib
import pandas as pd
from ratelimit import limits

In [84]:
# First we'll create a "Auth" object. Please, replace this with your credentials:

"""
- YOUR_ACCOUNT_ID
- YOUR_SETTINGS_ID
- YOUR_API_KEY
"""

accountId = "YOUR_ACCOUNT_ID"
settingID = "YOUR_SETTINGS_ID"
api_key = "YOUR_API_KEY"

def makeAuth():
    seconds = time.time()
    timestamp = int(seconds * 1000)
    token = str(accountId) + ":" + str(settingID) + ":" + str(timestamp) + api_key
    hash = hashlib.sha512(token.encode('utf-8')).hexdigest()
    auth = str(accountId) + ":" + str(settingID) + ":" + str(timestamp) + ":" + str(hash)
    return auth

authHash = makeAuth()

In [85]:
# For further needs we'll get the current list of operators names with ID's from the account 

url_operators = "https://api.boldchat.com/aid/YOUR_ACCOUNT_ID/data/rest/json/v1/getOperators?auth=" + authHash
response_operator = requests.get(url_operators)
operator_data = response_operator.json()

In [86]:
#Convert Operator JSON data into Pandas dataframe

operator_list = list(operator_data.values())[2]

loginID = []
UserName = []
ChatName = [] 
Name = []

for i in range(len(operator_list)):
    loginID.append(operator_list[i]['LoginID'])
    UserName.append(operator_list[i]['UserName'])
    ChatName.append(operator_list[i]['ChatName'])
    Name.append(operator_list[i]['Name'])

column_names_operator = ["login_id", "user_name", "chat_name", "name"]

operator_names = pd.DataFrame(columns = column_names_operator)

operator_names["login_id"] = loginID
operator_names["user_name"] = UserName
operator_names["chat_name"] = ChatName
operator_names["name"] = Name
operator_names["login_id"] = operator_names["login_id"].astype("int64")

In [87]:
#Create dictionary for replacement of OperatorID to Operator Name

operator_names["name"] = operator_names["name"].str.strip()

replace_dict_operators = pd.Series(operator_names.name.values, index = operator_names.login_id).to_dict()

In [88]:
#Get data for selected account folders + selected date. We'll gett all pages from API for choosen date.  

final_data_return = []
folders_id_list =[]
i = 0

#You need to replace this for YOUR account folder names and ID's
account_folders = {"FOLDER_1":"FOLDER_1_ID", "FOLDER_2":"FOLDER_2_ID", "FOLDER_3":"FOLDER_3_ID", "FOLDER_4":"FOLDER_4_ID"}

for key, values in account_folders.items():
    folders_id_list.append(values)

def records_limit():
    for i in folders_id_list: #we'll get data for each folder, one by one
        #below we get first page data from API and place it prepared list
        url_api = "https://api.boldchat.com/aid/YOUR_ACCOUNT_ID/data/rest/json/v2/getInactiveChats?FolderID=" + i + "&FromDate=2020-07-06T00:00:00&ToDate=2020-07-21T00:00:00&auth=" + authHash
        response_first = requests.get(url_api)
        page1 = response_first.json()
        final_data_return.append(page1)
        while list(final_data_return[-1].keys())[-1] == "NextPage": #here we check if the last page of our final list contains value for "Next Page", if it is - we continue to request data by changing the value of requested url
            next_page_value = list(final_data_return[-1].values())[3]
            url_api_next = "https://api.boldchat.com/aid/YOUR_ACCOUNT_ID/data/rest/json/v2/getInactiveChats?FolderID=" + i + "&NextPage=" + next_page_value + "&auth=" + authHash
            response_next = requests.get(url_api_next)
            page2 = response_next.json()
            final_data_return.append(page2)
        i =+ 1    
    print("Download complete. Total number of pages: " + str(len(final_data_return)))

In [89]:
records_limit()

Download complete. Total number of pages: 60


In [90]:
#Converting JSON data to dataframe

folder_id = []
operator = []
answered = []
ended = []
chat_name = []
chat_status_type = []
visitor_message_count = []

for g in range(len(final_data_return)):
    work_dictionary = final_data_return[g]
    for i in range(len(work_dictionary['Data'])):
        folder_id.append(work_dictionary['Data'][i]['FolderID'])
        operator.append(work_dictionary['Data'][i]['OperatorID'])
        answered.append(work_dictionary['Data'][i]['Answered'])
        ended.append(work_dictionary['Data'][i]['Ended'])
        chat_status_type.append(work_dictionary['Data'][i]['ChatStatusType'])
        visitor_message_count.append(work_dictionary['Data'][i]['VisitorMessageCount'])

column_names = ["folderid", "foldername", "operatorid", "operatorname", "team", "chatstatustype", "visitormessagecount", "answered", "ended", "time_difference", "date"]

chat_handling_time = pd.DataFrame(columns = column_names)

chat_handling_time["folderid"] = folder_id
chat_handling_time["operatorid"] = operator
chat_handling_time["answered"] = answered
chat_handling_time["ended"] = ended
chat_handling_time["chatstatustype"] = chat_status_type
chat_handling_time["visitormessagecount"] = visitor_message_count
chat_handling_time["foldername"] = folder_id
chat_handling_time["operatorname"] = operator

In [92]:
#Saving recieved data in CSV format

chat_handling_time.to_csv("Chat Handling time for 01-10 of July.csv", index=False)